<a href="https://colab.research.google.com/github/Tymass/chess-player/blob/yolo-training/YOLOv9_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv9 Quantization based on [this](https://docs.openvino.ai/2024/notebooks/yolov9-optimization-with-output.html) article.

### INSTALLATIONS

In [ ]:
!pip install torch torchvision
!pip install ultralytics
!pip install roboflow

import platform
if platform.system() != "Windows":
    %pip install -q "matplotlib>=3.4"
else:
    %pip install -q "matplotlib>=3.4,<3.7"

!pip install -q "nncf>=2.8.1"
!pip install -q "openvino>=2023.3.0" "opencv-python" "seaborn" "pandas" "scikit-learn" "torch" "torchvision"  --extra-index-url https://download.pytorch.org/whl/cpu


### IMPORTS

In [ ]:
from roboflow import Roboflow
import ipywidgets as widgets
import sys
from pathlib import Path
import torch
import nncf

import openvino as ov

import numpy as np
from PIL import Image

from collections import namedtuple
import yaml

from typing import List, Tuple
import time
import os

from google.colab import userdata

### CLONE YOLOv9 REPO

In [ ]:
sys.path.append("../utils")

if not Path('yolov9').exists():
    !git clone https://github.com/WongKinYiu/yolov9
%cd yolov9

from models.experimental import attempt_load
from models.yolo import Detect, DualDDetect
from utils.general import yaml_save, yaml_load
from utils.augmentations import letterbox
from utils.dataloaders import create_dataloader
from utils.general import colorstr
from utils.general import scale_boxes, non_max_suppression
from utils.plots import Annotator, colors

### FUNCTION DEFINITIONS

In [3]:
def preprocess_image(img0: np.ndarray):
    """
    Preprocess image according to YOLOv9 input requirements.
    Takes image in np.array format, resizes it to specific size using letterbox resize, converts color space from BGR (default in OpenCV) to RGB and changes data layout from HWC to CHW.

    Parameters:
      img0 (np.ndarray): image for preprocessing
    Returns:
      img (np.ndarray): image after preprocessing
      img0 (np.ndarray): original image
    """
    # resize
    img = letterbox(img0, auto=False)[0]

    # Convert
    img = img.transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    return img, img0

def prepare_input_tensor(image: np.ndarray):
    """
    Converts preprocessed image to tensor format according to YOLOv9 input requirements.
    Takes image in np.array format with unit8 data in [0, 255] range and converts it to torch.Tensor object with float data in [0, 1] range

    Parameters:
      image (np.ndarray): image for conversion to tensor
    Returns:
      input_tensor (torch.Tensor): float tensor ready to use for YOLOv9 inference
    """
    input_tensor = image.astype(np.float32)  # uint8 to fp16/32
    input_tensor /= 255.0  # 0 - 255 to 0.0 - 1.0

    if input_tensor.ndim == 3:
        input_tensor = np.expand_dims(input_tensor, 0)
    return input_tensor

def transform_fn(data_item):
    """
    Quantization transform function. Extracts and preprocess input data from dataloader item for quantization.
    Parameters:
       data_item: Tuple with data item produced by DataLoader during iteration
    Returns:
        input_tensor: Input data for quantization
    """
    img = data_item[0].numpy()
    input_tensor = prepare_input_tensor(img)
    return input_tensor

def draw_boxes(predictions: np.ndarray, input_shape: Tuple[int], image: np.ndarray, names: List[str]):
    """
    Utility function for drawing predicted bounding boxes on image
    Parameters:
        predictions (np.ndarray): list of detections with (n,6) shape, where n - number of detected boxes in format [x1, y1, x2, y2, score, label]
        image (np.ndarray): image for boxes visualization
        names (List[str]): list of names for each class in dataset
        colors (Dict[str, int]): mapping between class name and drawing color
    Returns:
        image (np.ndarray): box visualization result
    """
    if not len(predictions):
        return image

    annotator = Annotator(image, line_width=1, example=str(names))
    # Rescale boxes from input size to original image size
    predictions[:, :4] = scale_boxes(input_shape[2:], predictions[:, :4], image.shape).round()

    # Write results
    for *xyxy, conf, cls in reversed(predictions):
        label = f'{names[int(cls)]} {conf:.2f}'
        annotator.box_label(xyxy, label, color=colors(int(cls), True))
    return image

def detect(model: ov.Model, image_path: Path, conf_thres: float = 0.25, iou_thres: float = 0.45, classes: List[int] = None, agnostic_nms: bool = False):
    """
    OpenVINO YOLOv9 model inference function. Reads image, preprocess it, runs model inference and postprocess results using NMS.
    Parameters:
        model (Model): OpenVINO compiled model.
        image_path (Path): input image path.
        conf_thres (float, *optional*, 0.25): minimal accepted confidence for object filtering
        iou_thres (float, *optional*, 0.45): minimal overlap score for removing objects duplicates in NMS
        classes (List[int], *optional*, None): labels for prediction filtering, if not provided all predicted labels will be used
        agnostic_nms (bool, *optional*, False): apply class agnostic NMS approach or not
    Returns:
       pred (List): list of detections with (n,6) shape, where n - number of detected boxes in format [x1, y1, x2, y2, score, label]
       orig_img (np.ndarray): image before preprocessing, can be used for results visualization
       inpjut_shape (Tuple[int]): shape of model input tensor, can be used for output rescaling
    """
    if isinstance(image_path, np.ndarray):
        img = image_path
    else:
        img = np.array(Image.open(image_path))
    preprocessed_img, orig_img = preprocess_image(img)
    input_tensor = prepare_input_tensor(preprocessed_img)
    predictions = torch.from_numpy(model(input_tensor)[0])
    pred = non_max_suppression(predictions, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)
    return pred, orig_img, input_tensor.shape

def transform_fn(data_item):
    """
    Quantization transform function. Extracts and preprocess input data from dataloader item for quantization.
    Parameters:
       data_item: Tuple with data item produced by DataLoader during iteration
    Returns:
        input_tensor: Input data for quantization
    """
    img = data_item[0].numpy()
    input_tensor = prepare_input_tensor(img)
    return input_tensor


### DOWNLOAD DATASET

In [ ]:
API_key = userdata.get('API_key')
rf = Roboflow(api_key=API_key)
project = rf.workspace("tymek-byrwa-1p3fh").project("chesspiecesdetection-y9ljv")
project_version = 6
version = project.version(project_version)
dataset = version.download("yolov9")

YOU NEED TO UPLOAD WEIGHTS TO */content/yolov9/model*

In [6]:
DATA_DIR = Path(f'/content/yolov9/chessPiecesDetection-{project_version}/')
MODEL_DIR = Path("/content/yolov9/model/")
MODEL_DIR.mkdir(exist_ok=True)
DATA_DIR.mkdir(exist_ok=True)

### LOAD MODEL

In [ ]:
weights = MODEL_DIR / "best.pt"
ov_model_path = MODEL_DIR / weights.name.replace(".pt", "_openvino_model") / weights.name.replace(".pt", ".xml")


if not ov_model_path.exists():
    model = attempt_load(weights, device="cpu", inplace=True, fuse=True)
    metadata = {'stride': int(max(model.stride)), 'names': model.names}

    model.eval()
    for k, m in model.named_modules():
        if isinstance(m, (Detect, DualDDetect)):
            m.inplace = False
            m.dynamic = True
            m.export = True

    example_input = torch.zeros((1, 3, 640, 640))
    model(example_input)

    ov_model = ov.convert_model(model, example_input=example_input)

    # specify input and output names for compatibility with yolov9 repo interface
    ov_model.outputs[0].get_tensor().set_names({"output0"})
    ov_model.inputs[0].get_tensor().set_names({"images"})
    ov.save_model(ov_model, ov_model_path)
    # save metadata
    yaml_save(ov_model_path.parent / weights.name.replace(".pt", ".yaml"), metadata)
else:
    metadata = yaml_load(ov_model_path.parent / weights.name.replace(".pt", ".yaml"))

### CREATE DATALOADER


In [11]:
data[TASK]

'chessPiecesDetection-6/valid/images'

In [ ]:
# read dataset config
DATA_CONFIG = DATA_DIR / 'data.yaml'
YOLO_PATH = Path('/content/yolov9/')
with open(DATA_CONFIG) as f:
    data = yaml.load(f, Loader=yaml.SafeLoader)

# Dataloader
TASK = 'val'  # path to train/val/test images
Option = namedtuple('Options', ['single_cls'])  # imitation of commandline provided options for single class evaluation
opt = Option(False)

# Specify dataset for accuracy control Qunatization
if TASK == 'test':
  DATA_PATH = str(DATA_DIR / data[TASK])
else:
  DATA_PATH = str(YOLO_PATH / data[TASK])

dataloader = create_dataloader(
    DATA_PATH, 640, 1, 32, opt, pad=0.5,
    prefix=colorstr(f'{TASK}: ')
)[0]


quantization_dataset = nncf.Dataset(dataloader, transform_fn)

### PERFORM QUANTIZATION

In [ ]:
ov_int8_model_path = MODEL_DIR / weights.name.replace(".pt","_int8_openvino_model") / weights.name.replace(".pt", "_int8.xml")

if not ov_int8_model_path.exists():
    quantized_model = nncf.quantize(ov_model, quantization_dataset, preset=nncf.QuantizationPreset.MIXED)

    ov.save_model(quantized_model, ov_int8_model_path)
    yaml_save(ov_int8_model_path.parent / weights.name.replace(".pt", "_int8.yaml"), metadata)

### LOAD QUANTIZED MODEL

In [ ]:
core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

quantized_model = core.read_model(ov_int8_model_path)

if device.value != "CPU":
    quantized_model.reshape({0: [1, 3, 640, 640]})

compiled_model = core.compile_model(quantized_model, device.value)

In [ ]:
device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

### TEST QUNATIZED MODEL

In [ ]:
boxes, image, input_shape = detect(compiled_model,"/content/yolov9/chessPiecesDetection-5/test/images/1714395096705_jpg.rf.b25180e3b0aa27d77b1dc05a27a70caa.jpg")
NAMES = metadata["names"]
image_with_boxes = draw_boxes(boxes[0], input_shape, image, NAMES)
# visualize results
Image.fromarray(image_with_boxes)

### Benchmarks

In [ ]:
!benchmark_app -m $ov_model_path -shape "[1,3,640,640]" -d $device.value -api async -t 15

In [ ]:
!benchmark_app -m $ov_int8_model_path -shape "[1,3,640,640]" -d $device.value -api async -t 15

In [ ]:
# Calculate predictions time
def predictions_time(path):
    try:
        if not os.path.isdir(path):
            print("Wrong path.")
            return
        files = os.listdir(path)

        for file in files:
            t1 = time.time()

            results = model.predict(path + '/' + file)

            t2 = time.time()
            dt = t2 - t1

            print(f"Detection time: {dt:.4f} seconds")


    except Exception as e:
        print("Wystąpił błąd:", e)

In [ ]:
predictions_time('/content/yolov9/chessPiecesDetection-6/test')